## MCP - Time

### a) Configure Server Parameters


In [3]:
from mcp import StdioServerParameters

server_params = StdioServerParameters(
    command="npx",
    args=["-y", "time-mcp"],
    env=None,
)


### b) List Tools


In [4]:
from mcp.client.stdio import stdio_client
from mcp import ClientSession

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()
        tools = await session.list_tools()

        # List available tools
        for tool in tools.tools:
            print(f"Tool: {tool.name}\nDescription: {tool.description}\n")
            for name, prop in tool.inputSchema["properties"].items():
                print(f"  Argument: {name}")
                print(f"  Type: {prop['type']}")
                if "description" in prop:
                    print(f"  Description: {prop['description']}")


Tool: current_time
Description: Get the current date and time.

  Argument: format
  Type: string
  Description: The format of the time, default is empty string
  Argument: timezone
  Type: string
  Description: The timezone of the time, IANA timezone name, e.g. Asia/Shanghai
Tool: relative_time
Description: Get the relative time from now.

  Argument: time
  Type: string
  Description: The time to get the relative time from now. Format: YYYY-MM-DD HH:mm:ss
Tool: days_in_month
Description: Get the number of days in a month. If no date is provided, get the number of days in the current month.

  Argument: date
  Type: string
  Description: The date to get the days in month. Format: YYYY-MM-DD
Tool: get_timestamp
Description: Get the timestamp for the time.

  Argument: time
  Type: string
  Description: The time to get the timestamp. Format: YYYY-MM-DD HH:mm:ss
Tool: convert_time
Description: Convert time between timezones.

  Argument: sourceTimezone
  Type: string
  Description: The s

#### i. Get Current Time


In [22]:
from mcp.client.stdio import stdio_client
from mcp import ClientSession

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        import os
        tz = os.environ.get('TZ') or "utc"
        resp = await session.call_tool("current_time",arguments={"format":"YYYY-MMM-DD HH:mm:ss","timezone":tz})
        print(resp)


meta=None content=[TextContent(type='text', text='Current UTC time is 2025-Jun-14 04:59:22, and the time in utc is 2025-Jun-14 04:59:22.', annotations=None)] isError=False success=True


---

## MCP - FileSystem

### a) Configure Server Parameters

In [1]:
from mcp import StdioServerParameters
server_params = StdioServerParameters(
    command="npx",
    args=[
        "-y",
        "@modelcontextprotocol/server-filesystem",
        "/tmp"
    ],
    env=None
)

### b) List Tools

In [2]:
from mcp.client.stdio import stdio_client
from mcp import ClientSession
async with stdio_client(server_params) as (read, write):
    async with ClientSession(
        read, write
    ) as session:

        # Initialize the connection
        await session.initialize()
        tools = await session.list_tools()

        # List available tools
        for tool in tools.tools:
            print(f"Tool: {tool.name}\nDescription: {tool.description}\n")
            for name,prop in tool.inputSchema["properties"].items():
                print(f"  Argument: {name}")
                print(f"  Type: {prop['type']}")
                if "description" in prop:
                    print(f"  Description: {prop['description']}")


Tool: read_file
Description: Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.

  Argument: path
  Type: string
Tool: read_multiple_files
Description: Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.

  Argument: paths
  Type: array
Tool: write_file
Description: Create a new file or completely overwrite an existing file with new content. Use with caution as it will overwrite existing files without warning. Handles text content with proper encoding. Only works within allowed directories.

  Argument: path


#### i. List allowed directories

In [3]:
from mcp.client.stdio import stdio_client
from mcp import ClientSession
async with stdio_client(server_params) as (read, write):
    async with ClientSession(
        read, write
    ) as session:

        # Initialize the connection
        await session.initialize()
        resp = await session.call_tool("list_allowed_directories")
        print(resp)



meta=None content=[TextContent(type='text', text='Allowed directories:\n/tmp', annotations=None)] isError=False


#### ii. List directory

In [5]:
from mcp.client.stdio import stdio_client
from mcp import ClientSession
async with stdio_client(server_params) as (read, write):
    async with ClientSession(
        read, write
    ) as session:

        # Initialize the connection
        await session.initialize()
        resp = await session.call_tool("list_directory",arguments={"path":"/tmp"})
        print(resp)



meta=None content=[TextContent(type='text', text='[DIR] .X11-unix\n[DIR] devcontainercli-kakkoii1337\n[DIR] devcontainers-1686179e-3599-4599-b59c-6768be3ec1cc1749612777360\n[DIR] devcontainers-1977e3f8-b3f3-4a0e-97fc-00a01cf413081749613340563\n[DIR] devcontainers-36b4afe3-1ef3-4c07-bc17-6297d7cebb851749613026450\n[DIR] devcontainers-e960a2d9-20e5-4051-9c68-cd8c9978a7151749613230615\n[FILE] dockerd.log\n[DIR] kakkoii1337-code-zsh\n[DIR] node-compile-cache\n[DIR] pyright-1602-wU3byH4KsNDw\n[DIR] python-languageserver-cancellation\n[FILE] remote-wsl-loc.txt\n[DIR] snap-private-tmp\n[DIR] ssh-XXXXXXIIGZNm\n[DIR] systemd-private-d936ba9b290e429aaeaedd09ea6ff704-systemd-logind.service-EPZX7z\n[DIR] systemd-private-d936ba9b290e429aaeaedd09ea6ff704-systemd-resolved.service-6p3Xdv\n[DIR] systemd-private-d936ba9b290e429aaeaedd09ea6ff704-systemd-timesyncd.service-kLFePw\n[DIR] user\n[FILE] vscode-distro-env.AuX4sW\n[FILE] vscode-distro-env.lWaPAa\n[FILE] vscode-ipc-16348fd2-5e05-4e92-a79a-9da14af

---

## MCP - Web

In [1]:
from mcp import StdioServerParameters
server_params = StdioServerParameters(
    command="uvx",
    args=[
        "gai-tools-web"
    ],
    env=None
)

In [2]:
from mcp.client.stdio import stdio_client
from mcp import ClientSession
async with stdio_client(server_params) as (read, write):
    async with ClientSession(
        read, write
    ) as session:

        # Initialize the connection
        await session.initialize()
        tools = await session.list_tools()

        # List available tools
        for tool in tools.tools:
            print(f"Tool: {tool.name}\nDescription: {tool.description}\n")
            for name,prop in tool.inputSchema["properties"].items():
                print(f"  Argument: {name}")
                print(f"  Type: {prop['type']}")
                if "description" in prop:
                    print(f"  Description: {prop['description']}")


Tool: scrape
Description: 
    Scrape a webpage and return the parsed response.

    Args:
        - url (str): The URL of the webpage to scrape.
    Returns:
        - source: Source of the parsed content
        - title: Title of the document
        - text: Main body of the document
        - authors: List of authors
        - summary: Summary of the document
        - keywords: Keywords associated with the document
        - categories: Categories the document belongs to
        - publish_date: Publication date of the document
        - length: Total length of the document in characters, computed from text
        - created_at: Creation timestamp of the parsed response
        - links: Dictionary of links related to the document
        - chunks: Optional list of text chunks parsed from the document
    

  Argument: url
  Type: string
Tool: search
Description: 
    The 'search' function is a powerful tool that allows the AI to gather external information from the internet using Go